## Perceptron for boolean functions

The perceptron is a simple model of a neuron. It takes a vector of inputs, multiplies each input by a weight, sums the results, and applies a step function (heavy-side function) to the sum. The output is 1 if the sum is greater than a threshold and 0 otherwise.

<img src="../../assets/unit_step.png" style="display: block; margin-left: auto; margin-right: auto; max-width:400px">

The perceptron can be used to implement boolean functions. For example, the AND function can be implemented with a perceptron with weights $ w_1 = 1, w_2 = 1 $ and bias $ b = -1.5 $.
Boolean | Bias | Weight
--- | --- | ---
AND | -1.5  | 1, 1
OR  | -0.5  | 1, 1
NOT | 0.5 | -1

Now based on inputs we will get the output as follows:

Input | AND | OR | XOR
--- | --- | --- | ---
0, 0 | 0 | 0 | 0
0, 1 | 0 | 1 | 1
1, 0 | 0 | 1 | 1
1, 1 | 1 | 1 | 0

In [31]:
import numpy as np

def unit_step(v: float) -> int:
	'''Heaviside step function'''
	if v >= 0:
		return 1
	else:
		return 0
	
def Perceptron(x: np.ndarray, w: np.ndarray, b: float) -> int:
	'''Perceptron function'''
	v = np.dot(w, x) + b
	y = unit_step(v)
	return y

def NOT_percep(x: int) -> int:
	'''
	NOT gate
	
	Examples:
		>>> NOT_percep(0)
		1
		>>> NOT_percep(1)
		0
	'''
	return Perceptron(x, w=-1, b=0.5)

def AND_percep(x: np.ndarray) -> int:
	'''
	AND gate

	Examples:
		>>> AND_percep(np.array([0, 0])) 
		0
		>>> AND_percep(np.array([0, 1]))
		0
		>>> AND_percep(np.array([1, 0]))
		0
		>>> AND_percep(np.array([1, 1]))
		1
	'''
	w = np.array([1, 1])
	b = -1.5
	return Perceptron(x, w, b)

def OR_percep(x: np.ndarray) -> int:
	'''
	OR gate

	Examples:
		>>> OR_percep(np.array([0, 0])) 
		0
		>>> OR_percep(np.array([0, 1]))
		1
		>>> OR_percep(np.array([1, 0]))
		1
		>>> OR_percep(np.array([1, 1]))
		1
	'''
	w = np.array([1, 1])
	b = -0.5
	return Perceptron(x, w, b)

def XOR_1_percep(x):
	'''
	XOR gate using AND, OR and NOT gates
	
	Examples:
		>>> XOR_1_percep(np.array([0, 0]))
		0
		>>> XOR_1_percep(np.array([0, 1]))
		1
		>>> XOR_1_percep(np.array([1, 0]))
		1
		>>> XOR_1_percep(np.array([1, 1]))
		0
	'''
	gate_1 = AND_percep(np.array([x[0], NOT_percep(x[1])]))
	gate_2 = AND_percep(np.array([NOT_percep(x[0]), x[1]]))
	return OR_percep(np.array([gate_1, gate_2]))

def XOR_2_percep(x):
	'''
	XOR gate using AND, OR and NOT gates

	Examples:
		>>> XOR_2_percep(np.array([0, 0]))
		0
		>>> XOR_2_percep(np.array([0, 1]))
		1
		>>> XOR_2_percep(np.array([1, 0]))
		1
		>>> XOR_2_percep(np.array([1, 1]))
		0
	'''
	gate_1 = AND_percep(x)
	gate_2 = OR_percep(x)
	gate_3 = NOT_percep(gate_1)
	return AND_percep(np.array([gate_2, gate_3]))

if __name__ == '__main__':
	import doctest
	if doctest.testmod().failed == 0:
		print('All tests passed')

All tests passed


In [32]:
def perceptron_train(x: np.ndarray, y: np.ndarray, a: float, epochs: int) -> tuple:
    '''
    Train a perceptron model

    Parameters:
        x (np.ndarray): input data
        y (np.ndarray): target data
        a (float): learning rate
        epochs (int): number of epochs
    
    Returns:
        tuple: weights and bias
    '''
    w = np.zeros(x.shape[1])
    b = 0
    
    for _ in range(epochs):
        for i in range(len(x)):
            y_pred = Perceptron(x[i], w, b)
            
            if y_pred != y[i]:
                w += a * (y[i] - y_pred) * x[i]
                b += a * (y[i] - y_pred)
    
    return w, b

How it was above mentioned, the XOR function cannot be implemented with a single perceptron.

In [47]:
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])

y = {
    'AND': np.array([0, 0, 0, 1]),
    'OR': np.array([0, 1, 1, 1]),
    'NAND': np.array([1, 1, 1, 0]),
    'NOR': np.array([1, 0, 0, 0]),
    'XOR': np.array([0, 1, 1, 0])
}

test = {
    'AND': [0, 0, 0, 1],
    'OR': [0, 1, 1, 1],
    'NAND': [1, 1, 1, 0],
    'NOR': [1, 0, 0, 0],
    'XOR': [0, 1, 1, 0]
}

for gate in y:
    w, b = perceptron_train(x, y[gate], a=0.1, epochs=10)
    output = []
    print(f'\n{gate} w: {w}, b: {b}')
    for i in range(len(x)):
        print(f'y_pred = {Perceptron(x[i], w, b)} with x = {x[i]}')
        output.append(Perceptron(x[i], w, b))
    
    print(f'{gate} output: {output} == {test[gate]} => {output == test[gate]}')


AND w: [0.2 0.1], b: -0.20000000000000004
y_pred = 0 with x = [0 0]
y_pred = 0 with x = [0 1]
y_pred = 0 with x = [1 0]
y_pred = 1 with x = [1 1]
AND output: [0, 0, 0, 1] == [0, 0, 0, 1] => True

OR w: [0.1 0.1], b: -0.1
y_pred = 0 with x = [0 0]
y_pred = 1 with x = [0 1]
y_pred = 1 with x = [1 0]
y_pred = 1 with x = [1 1]
OR output: [0, 1, 1, 1] == [0, 1, 1, 1] => True

NAND w: [-0.2 -0.1], b: 0.2
y_pred = 1 with x = [0 0]
y_pred = 1 with x = [0 1]
y_pred = 1 with x = [1 0]
y_pred = 0 with x = [1 1]
NAND output: [1, 1, 1, 0] == [1, 1, 1, 0] => True

NOR w: [-0.1 -0.1], b: 0.0
y_pred = 1 with x = [0 0]
y_pred = 0 with x = [0 1]
y_pred = 0 with x = [1 0]
y_pred = 0 with x = [1 1]
NOR output: [1, 0, 0, 0] == [1, 0, 0, 0] => True

XOR w: [-0.1  0. ], b: 0.0
y_pred = 1 with x = [0 0]
y_pred = 1 with x = [0 1]
y_pred = 0 with x = [1 0]
y_pred = 0 with x = [1 1]
XOR output: [1, 1, 0, 0] == [0, 1, 1, 0] => False
